In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


In [2]:
file_path = "/Users/siddharth/Downloads/modelnet10.npz"
data = np.load(file_path, allow_pickle=True)
train_voxel = data["train_voxel"] # Training 3D voxel samples
test_voxel = data["test_voxel"] # Test 3D voxel samples
train_labels = data["train_labels"] # Training labels (integers from 0 to 9)
test_labels = data["test_labels"] # Test labels (integers from 0 to 9)
class_map = data["class_map"] # Dictionary mapping the labels to their class names.

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, in_channels=200, out_dim=64, out_channels=1):
        super(Generator, self).__init__()
        self.in_channels = in_channels
        self.out_dim = out_dim
        
        # Define the generator architecture
        self.layers = nn.Sequential(
            # Fully connected layer to project noise vector to higher-dimensional space
            nn.Linear(in_channels, 512 * out_dim * out_dim * out_dim),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512 * out_dim * out_dim * out_dim),
            
            # Reshape to a 4D tensor
            nn.Unflatten(1, (512, out_dim, out_dim, out_dim)),
            
            # First convolutional layer
            nn.ConvTranspose3d(
                in_channels=512, out_channels=256, kernel_size=4,
                stride=1, padding=0, bias=False
            ),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(256),
            
            # Second convolutional layer
            nn.ConvTranspose3d(
                in_channels=256, out_channels=128, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(128),
            
            # Third convolutional layer
            nn.ConvTranspose3d(
                in_channels=128, out_channels=64, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.ReLU(inplace=True),
            nn.BatchNorm3d(64),
            
            # Fourth convolutional layer
            nn.ConvTranspose3d(
                in_channels=64, out_channels=out_channels, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Instantiate the generator
generator = Generator()

# Print the generator architecture
print(generator)


In [ ]:
import torch
import torch.nn as nn

class Discriminator(nn.Module):
    def __init__(self, in_channels=1):
        super(Discriminator, self).__init__()
        
        # Define the discriminator architecture
        self.layers = nn.Sequential(
            # First convolutional layer
            nn.Conv3d(
                in_channels=in_channels, out_channels=64, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(64),
            
            # Second convolutional layer
            nn.Conv3d(
                in_channels=64, out_channels=128, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(128),
            
            # Third convolutional layer
            nn.Conv3d(
                in_channels=128, out_channels=256, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(256),
            
            # Fourth convolutional layer
            nn.Conv3d(
                in_channels=256, out_channels=512, kernel_size=4,
                stride=2, padding=1, bias=False
            ),
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm3d(512),
            
            # Fifth convolutional layer
            nn.Conv3d(
                in_channels=512, out_channels=1, kernel_size=4,
                stride=1, padding=0, bias=False
            ),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

# Instantiate the discriminator
discriminator = Discriminator()

# Print the discriminator architecture
print(discriminator)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the Generator and Discriminator classes (as previously defined)

# Define a function to create random noise
def generate_noise(batch_size, noise_dim):
    return torch.rand(batch_size, noise_dim)

# Create the Generator and Discriminator instances
generator = Generator(in_channels=512, out_dim=64, out_channels=1, noise_dim=200)
discriminator = Discriminator()

# Define the loss and optimizers
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer_generator = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=0.0002)

# Training parameters
num_epochs = 3
batch_size = 64  # You can adjust this within the range of 10 to 100

# Assuming you have loaded your dataset into train_voxel, convert it to PyTorch tensors
train_voxel = torch.tensor(train_voxel, dtype=torch.float32)

# Create a DataLoader for batch training
train_dataset = TensorDataset(train_voxel)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for batch_data in train_loader:
        real_data = batch_data[0]  # Assuming your data is structured as a single tensor
        batch_size = real_data.size(0)
        
        # Train the Discriminator
        optimizer_discriminator.zero_grad()
        noise = generate_noise(batch_size, noise_dim)
        fake_data = generator(noise)
        
        # Real data target
        target_real = Variable(torch.ones(batch_size, 1))
        loss_real = criterion(discriminator(real_data), target_real)
        
        # Fake data target
        target_fake = Variable(torch.zeros(batch_size, 1))
        loss_fake = criterion(discriminator(fake_data), target_fake)
        
        loss_discriminator = loss_real + loss_fake
        loss_discriminator.backward()
        optimizer_discriminator.step()
        
        # Train the Generator
        optimizer_generator.zero_grad()
        noise = generate_noise(batch_size, noise_dim)
        fake_data = generator(noise)
        target_real = Variable(torch.ones(batch_size, 1))
        loss_generator = criterion(discriminator(fake_data), target_real)
        loss_generator.backward()
        optimizer_generator.step()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Discriminator Loss: {loss_discriminator.item()}, Generator Loss: {loss_generator.item()}")

# Save your generator and discriminator models if needed
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")
